# Notes

v1.5: added blade pitch
v1.4: updated turb generation and filter files function

# Import

In [2]:
import os
import functions
import numpy as np
import pandas as pd
from scipy.interpolate import interp1d
import jmespath

from windio_converter.io import HAWC2_dict
from windio_converter.io import HTC_dict

# Inputs

In [3]:
template_file = "IEA_15MW_RWT_UMaineSemi"
base_filename = "15MW_UMaineSemi"
iea15_manual_repo_fpath =  "C:/CRPO/02_Running_area/DTU_Thesis/IEA-15-240-RWT/HAWC2/IEA-15-240-RWT-UMaineSemi"
iea15_doc_fpath = "C:/CRPO/02_Running_area/DTU_Thesis/IEA-15-240-RWT/Documentation"
iea15_file = "IEA-15-240-RWT_tabular.xlsx"

In [4]:
iea15_data = pd.read_excel(os.path.join(iea15_doc_fpath,iea15_file),sheet_name="Rotor Performance")

In [5]:
new_data = "operation.txt"
iea15_data2 = pd.read_csv(os.path.join(iea15_doc_fpath,new_data), sep=';')
iea15_data2.columns = iea15_data2.columns.str.strip()
iea15_data2["Rotor Speed [rads]"] = iea15_data2["Rotor Speed [rpm]"]*(2 * np.pi / 60)

# General Functions

In [6]:
def interpolate_value(x_var,y_var,df,chosen_x_value):
    """
    Interpolates the y-value for a given x-value based on a dataframe.

    Parameters:
    x_var (str): The name of the column with x values (independent variable).
    y_var (str): The name of the column with y values (dependent variable).
    df (pd.DataFrame): The dataframe containing the data. 
    chosen_x_value (float): The x-value for which to estimate the y-value.

    Returns:
    float: The interpolated y-value.
    """
    # Ensure the dataframe is sorted by the x_column
    df = df.sort_values(by=x_var)
    # Create an interpolation function
    interp_func = interp1d(df[x_var], df[y_var], kind="linear", fill_value="extrapolate")
    # Interpolate the value
    interpolated_value = interp_func(chosen_x_value)
    return interpolated_value

In [7]:
def load_iea15_template(template_file, base_filename, repo_path):
    """
    Load the IEA 15MW HAWC2 template and return the dictionary.
    
    Args:
        template_file (str): Name of the HAWC2 template file without extension.
        base_filename (str): Start of all new files without extension.
        repo_path (str): Path to the repository containing the template HAWC2 files.

    Returns:
        dict: The resulting IEA 15MW HAWC2 dictionary of the template model ready to be manipulated.
    """
    iea15_H2 = HAWC2_dict().read_hawc2(repo_path, "htc/%s.htc"%template_file)
    iea15_H2_dict = iea15_H2.copy()
    iea15_H2_dict['htc_filename']='./htc/%s.htc'%base_filename
    iea15_H2_dict["htc"]["simulation"]["visualization"] = "./visualization/%s.hdf5"%base_filename
    iea15_H2_dict["htc"]["simulation"]["logfile"] = "./log/%s.log"%base_filename
    iea15_H2_dict["htc"]["output"]["filename"] = "./res/%s.hdf5"%base_filename
    iea15_H2_dict["htc"]["simulation"]["animation"] = "./ani/%s.dat"%base_filename
    iea15_H2_dict["htc"]["new_htc_structure"]["ext_sys"][0]["data"][0][1] = './HydroData/IEA-15-240-RWT-UMaineSemi'
    return iea15_H2_dict

In [8]:
def create_file_names(iea15_H2_dict, file_name_add, prepend_str=None):
    """
    Create new file names for HAWC2 files and return the dictionary.
    
    Args:
        iea15_H2_dict (HAWC2_dict): HAWC2 dictionary object
        file_name_add (str): string of the descriptive text to add
        prepend_str (str)(optional): If given, prepends the file names with this string

    Returns:
        dict: The resulting IEA 15MW HAWC2 dictionary with adjusted file names.
    """
    # add str to end of file names
    iea15_H2_dict["htc_filename"] = ".".join(iea15_H2_dict["htc_filename"].split(".")[:-1])+"%s.htc"%file_name_add
    iea15_H2_dict["htc"]["simulation"]["visualization"] = ".".join(iea15_H2_dict["htc"]["simulation"]["visualization"].split(".")[:-1])+"%s.hdf5"%file_name_add
    iea15_H2_dict["htc"]["simulation"]["logfile"] = ".".join(iea15_H2_dict["htc"]["simulation"]["logfile"].split(".")[:-1])+"%s.log"%file_name_add
    iea15_H2_dict["htc"]["output"]["filename"] = ".".join(iea15_H2_dict["htc"]["output"]["filename"].split(".")[:-1])+"%s.hdf5"%file_name_add #.hdf5 is doubled but it works
    iea15_H2_dict["htc"]["simulation"]["animation"] = ".".join(iea15_H2_dict["htc"]["simulation"]["animation"].split(".")[:-1])+"%s.dat"%file_name_add
    # Optional: prepend str to beginning of file names
    if prepend_str:
        iea15_H2_dict["htc_filename"] = iea15_H2_dict["htc_filename"].replace("htc/","htc/%s")%prepend_str
        iea15_H2_dict["htc"]["simulation"]["visualization"] = iea15_H2_dict["htc"]["simulation"]["visualization"].replace("visualization/","visualization/%s")%prepend_str
        iea15_H2_dict["htc"]["simulation"]["logfile"] = iea15_H2_dict["htc"]["simulation"]["logfile"].replace("log/","log/%s")%prepend_str
        iea15_H2_dict["htc"]["output"]["filename"] = iea15_H2_dict["htc"]["output"]["filename"].replace("res/","res/%s")%prepend_str
        iea15_H2_dict["htc"]["simulation"]["animation"] = iea15_H2_dict["htc"]["simulation"]["animation"].replace("ani/","ani/%s")%prepend_str
    return iea15_H2_dict

In [9]:
def remove_viz_ani(iea15_H2_dict):
    iea15_H2_dict["htc"]["simulation"].update({"visualization":None}) # removes heavy files
    iea15_H2_dict["htc"]["simulation"].update({"animation":None}) # removes heavy files
    return iea15_H2_dict

# HAWC2 Model Adjustments

In [10]:
def add_damping(iea15_H2_dict):
    iea15_H2_dict["htc"].update({"force": {}})
    iea15_H2_dict["htc"]["force"].update(
        {"dll" : 
         [{"name":"fext", 
        "dll": "./extforce/ForceTimeSeries.dll",
        "init": ["forcetimeseries_init", "./extforce/damp4_init_100.txt"],
        "update": "dampveltimeseries_update",
        "output": "output",
        "mbdy" : "ifb",
        "node" : 1
        }]}
    )
    return iea15_H2_dict

In [11]:
def adj_blade_output(iea15_H2_dict):
    #iea15_H2_dict["htc"]["output"]["mbdy"][13] = HAWC2_dict(["state","pos", "blade1", 33, 1.0, "global", "#", "gl", "blade", 1, "tip", "pos"])
    iea15_H2_dict["htc"]["output"]["mbdy"][13] = ["state","pos", "blade1", 33, 1.0, "global", "#", "gl", "blade", 1, "tip", "pos"]
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["state","pos", "blade2", 33, 1.0, "global", "#", "gl", "blade", 2, "tip", "pos"])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["state","pos", "blade3", 33, 1.0, "global", "#", "gl", "blade", 3, "tip", "pos"])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["state","pos", "blade1", 33, 1.0, "blade1", "#", "local", "blade", 1, "tip", "pos"])
    #cannot figure out how to update that item with new values. it works to add a new one using append.
    #this should be moved to the original call. It will need to be changed for all.
    return iea15_H2_dict      

In [12]:

def adj_blade_nbodies(iea15_H2_dict):
    """
    changes number of bodies in the blade
    """
    b1 = jmespath.search(
    "new_htc_structure.main_body[?contains(name,'blade1')]|[0]",
    iea15_H2_dict["htc"].as_dict()
    )
    i_ = iea15_H2_dict["htc"]["new_htc_structure"]["main_body"].index(b1)
    nbodies = iea15_H2_dict["htc"]["new_htc_structure"]["main_body"][i_]["c2_def"]["nsec"] - 1 #searches for nsec and creates nbodies - 1
    iea15_H2_dict["htc"]["new_htc_structure"]["main_body"][i_]["nbodies"] = nbodies


In [13]:
def add_aero_sec_int_forces(iea15_H2_dict):
    iea15_H2_dict["htc"]["output"]["aero"].append(["secforce", 1, 1, 10])
    iea15_H2_dict["htc"]["output"]["aero"].append(["secforce", 1, 2, 10])
    iea15_H2_dict["htc"]["output"]["aero"].append(["secforce", 1, 3, 10])
    iea15_H2_dict["htc"]["output"]["aero"].append(["secforce", 1, 1, 50])
    iea15_H2_dict["htc"]["output"]["aero"].append(["secforce", 1, 2, 50])
    iea15_H2_dict["htc"]["output"]["aero"].append(["secforce", 1, 3, 50])
    iea15_H2_dict["htc"]["output"]["aero"].append(["secforce", 1, 1, 100])
    iea15_H2_dict["htc"]["output"]["aero"].append(["secforce", 1, 2, 100])
    iea15_H2_dict["htc"]["output"]["aero"].append(["secforce", 1, 3, 100])
    iea15_H2_dict["htc"]["output"]["aero"].append(["secforce", 1, 1, 117])
    iea15_H2_dict["htc"]["output"]["aero"].append(["secforce", 1, 2, 117])
    iea15_H2_dict["htc"]["output"]["aero"].append(["secforce", 1, 3, 117])
    iea15_H2_dict["htc"]["output"]["aero"].append(["int_force", 1, 1, 1, 10])
    iea15_H2_dict["htc"]["output"]["aero"].append(["int_force", 1, 1, 2, 10])
    iea15_H2_dict["htc"]["output"]["aero"].append(["int_force", 1, 1, 3, 10])
    iea15_H2_dict["htc"]["output"]["aero"].append(["int_force", 1, 1, 1, 50])
    iea15_H2_dict["htc"]["output"]["aero"].append(["int_force", 1, 1, 2, 50])
    iea15_H2_dict["htc"]["output"]["aero"].append(["int_force", 1, 1, 3, 50])
    iea15_H2_dict["htc"]["output"]["aero"].append(["int_force", 1, 1, 1, 100])
    iea15_H2_dict["htc"]["output"]["aero"].append(["int_force", 1, 1, 2, 100])
    iea15_H2_dict["htc"]["output"]["aero"].append(["int_force", 1, 1, 3, 100])
    iea15_H2_dict["htc"]["output"]["aero"].append(["int_force", 1, 1, 1, 117])
    iea15_H2_dict["htc"]["output"]["aero"].append(["int_force", 1, 1, 2, 117])
    iea15_H2_dict["htc"]["output"]["aero"].append(["int_force", 1, 1, 3, 117])

    return iea15_H2_dict

In [14]:
def add_blade_pos_sensor(iea15_H2_dict):
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["statevec_new", "blade1", "c2def", "blade1", "absolute", 30, 1.0, 0.0, 0.0])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["statevec_new", "blade1", "c2def", "blade1", "absolute", 60, 1.0, 0.0, 0.0])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["statevec_new", "blade1", "c2def", "blade1", "absolute", 90, 1.0, 0.0, 0.0])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["statevec_new", "blade1", "c2def", "blade1", "absolute", 110, 1.0, 0.0, 0.0])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["statevec_new", "blade2", "c2def", "blade2", "absolute", 30, 1.0, 0.0, 0.0])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["statevec_new", "blade2", "c2def", "blade2", "absolute", 60, 1.0, 0.0, 0.0])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["statevec_new", "blade2", "c2def", "blade2", "absolute", 90, 1.0, 0.0, 0.0])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["statevec_new", "blade2", "c2def", "blade2", "absolute", 110, 1.0, 0.0, 0.0])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["statevec_new", "blade3", "c2def", "blade3", "absolute", 30, 1.0, 0.0, 0.0])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["statevec_new", "blade3", "c2def", "blade3", "absolute", 60, 1.0, 0.0, 0.0])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["statevec_new", "blade3", "c2def", "blade3", "absolute", 90, 1.0, 0.0, 0.0])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["statevec_new", "blade3", "c2def", "blade3", "absolute", 110, 1.0, 0.0, 0.0])
    
    return iea15_H2_dict

In [15]:
def add_shaft_force_sensor(iea15_H2_dict):
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["forcevec", "shaft", 1, 1, "shaft"])
    iea15_H2_dict["htc"]["output"]["mbdy"].append(["forcevec", "shaft", 1, 1, "global"])
    return iea15_H2_dict

In [16]:
def adj_fix_rotor(iea15_H2_dict):
    file_name_add = "_Fix_rotor"
    iea15_H2_dict = create_file_names(iea15_H2_dict,file_name_add)
    iea15_H2_dict["htc"]["new_htc_structure"]["constraint"].update(
        {"fix0": [
        #{"mbdy" : ["tower"]},
        {"mbdy" : ["towertop"]}
        ]})
    #adjust outputs for blades 
    return iea15_H2_dict


In [17]:
def adj_fix_tower_bottom(iea15_H2_dict):
    file_name_add = "_Fix_tow_bot"
    iea15_H2_dict = create_file_names(iea15_H2_dict,file_name_add)
    iea15_H2_dict["htc"]["new_htc_structure"]["constraint"].update(
        {"fix0": [
        {"mbdy" : ["tower"]}
        ]})
    
    return iea15_H2_dict

In [18]:
def adj_stiff_bld_new(iea15_H2_dict):
    file_name_add = "_Stiff_bld"
    iea15_H2_dict = create_file_names(iea15_H2_dict,file_name_add)
    iea15_H2_dict["htc"]["new_htc_structure"]["main_body"][8]["timoschenko_input"]["filename"] = ".".join(
        iea15_H2_dict["htc"]["new_htc_structure"]["main_body"][8]["timoschenko_input"]["filename"].split(".")[:-1]
        )+"%s.st"%file_name_add
    iea15_H2_dict["st"]["blade1"][0][0]["E"] = [el*1e7 for el in iea15_H2_dict["st"]["blade1"][0][0]["E"]]
    iea15_H2_dict["st"]["blade1"][0][0]["G"] = [el*1e7 for el in iea15_H2_dict["st"]["blade1"][0][0]["G"]]
    return iea15_H2_dict

In [24]:
def adj_stiff_tors_bld_new(iea15_H2_dict):
    file_name_add = "_Stiff_tors_bld"
    iea15_H2_dict = create_file_names(iea15_H2_dict,file_name_add)
    iea15_H2_dict["htc"]["new_htc_structure"]["main_body"][8]["timoschenko_input"]["filename"] = ".".join(
        iea15_H2_dict["htc"]["new_htc_structure"]["main_body"][8]["timoschenko_input"]["filename"].split(".")[:-1]
        )+"%s.st"%file_name_add
    iea15_H2_dict["st"]["blade1"][0][0]["E"] = [el*1e7 for el in iea15_H2_dict["st"]["blade1"][0][0]["E"]]
    iea15_H2_dict["st"]["blade1"][0][0]["G"] = [el*1e7 for el in iea15_H2_dict["st"]["blade1"][0][0]["G"]]
    iea15_H2_dict["st"]["blade1"][0][0]["K"] = [el*1e7 for el in iea15_H2_dict["st"]["blade1"][0][0]["K"]]
    return iea15_H2_dict

In [19]:
def update_tower_nbodies(iea15_H2_dict):
    iea15_H2_dict["htc"]["new_htc_structure"]["main_body"][1]["nbodies"] = 10

# HAWC2 Generate cases

In [20]:
def generate_steady_state_wo_ctrl(iea15_H2_dict, wsp, prepend_str):
    file_name_add = "_Wo_ctrl_steady_%sms"%wsp
    iea15_H2_dict = create_file_names(iea15_H2_dict,file_name_add, prepend_str)
    iea15_H2_dict["htc"]["simulation"]["time_stop"] = 200 
    iea15_H2_dict["htc"]["simulation"]["solvertype"] = 2
    iea15_H2_dict["htc"]["simulation"]["intial_condition"] = 1 
    # need to set a wsp and also change several inputs to change bearings, use omega value, remove controller, change output.
    iea15_H2_dict["htc"]["wind"]["wsp"] = wsp
    iea15_H2_dict["htc"].update({"dll":None})
    # look up rotor speed for wsp
    rotor_speed_rad = interpolate_value("Wind [m/s]", "Rotor Speed [rpm]", iea15_data2, wsp)* (2 * 3.141592653589793 / 60)
    blade_pitch = interpolate_value("Wind [m/s]", "Pitch [deg]", iea15_data2, wsp)
    # change orientations
    omega = rotor_speed_rad # need to get this in list with wsp
    iea15_H2_dict["htc"]["new_htc_structure"]["orientation"]["relative"][3]['mbdy2_ini_rotvec_d1'] = [0,0,-1,omega]
    iea15_H2_dict["htc"]["new_htc_structure"]["orientation"]["relative"][7]["mbdy2_eulerang"] =  [0,0, - blade_pitch]
    iea15_H2_dict["htc"]["new_htc_structure"]["orientation"]["relative"][8]["mbdy2_eulerang"] =  [0,0, - blade_pitch]
    iea15_H2_dict["htc"]["new_htc_structure"]["orientation"]["relative"][9]["mbdy2_eulerang"] =  [0,0, - blade_pitch]

    #several removals and additions of sections.
    #iea15_H2_dict["htc"]["new_htc_structure"]["constraint"]["bearing1"].update({"name":"shaft_rot":None}) #ideally want to call out the name of bearing to delete
    iea15_H2_dict["htc"]["new_htc_structure"]["constraint"].update(
        {"bearing1":None})
    iea15_H2_dict["htc"]["new_htc_structure"]["constraint"].update(
        {"bearing3": 
        [{"name":"shaft_rot", 
        "mbdy1": ["connector", "last"],
        "mbdy2": ["shaft", 1],
        "bearing_vector": [2, 0, 0, -1],
        "omegas": [omega]}]
        })
    #now i want to delete specify bearing 2 and replace with fix1
    iea15_H2_dict["htc"]["new_htc_structure"]["constraint"].update(
        {"bearing2":None}) 

    iea15_H2_dict["htc"]["new_htc_structure"]["constraint"]["fix1"].append(
        HTC_dict({
       "mbdy1": ["hub1", "last"],
       "mbdy2": ["blade1" , 1]})
       )                                                                
    iea15_H2_dict["htc"]["new_htc_structure"]["constraint"]["fix1"].append(
        HTC_dict({
       "mbdy1": ["hub2", "last"],
       "mbdy2": ["blade2" , 1]})
       )                                                                         
    iea15_H2_dict["htc"]["new_htc_structure"]["constraint"]["fix1"].append(
        HTC_dict({
       "mbdy1": ["hub3", "last"],
       "mbdy2": ["blade3" , 1]})
       )            

    #adjust output files:
    iea15_H2_dict["htc"]["output"]["constraint"].update(
        {1: None})
    iea15_H2_dict["htc"]["output"]["constraint"].update(
        {1: None})
    iea15_H2_dict["htc"]["output"]["constraint"].update(
        {1: None}) # this is done to get rid of 3 occurences.
    iea15_H2_dict["htc"]["output"]["constraint"].update({0 : ["bearing3","shaft_rot",2]}) # need to know its first instance (cannot call name)
    
    iea15_H2_dict["htc"]["output"].update(
        {"dll": None})
    #Add floater properties    
    iea15_H2_dict["htc"]["output"].update(
        {"esys":[["floater"], ["line1"], ["line2"], ["line3"]] 
         })

    return iea15_H2_dict
    
  

In [21]:
def generate_stepwind(iea15_H2_dict, wsp, prepend_str):
    file_name_add = "_stepwind_%sms"%wsp
    iea15_H2_dict = create_file_names(iea15_H2_dict,file_name_add, prepend_str)
    iea15_H2_dict["htc"]["simulation"]["time_stop"] = 2050 
    iea15_H2_dict["htc"]["simulation"]["solvertype"] = 2
    iea15_H2_dict["htc"]["simulation"]["intial_condition"] = 1 
    iea15_H2_dict["htc"]["wind"]["wsp"] = wsp
    iea15_H2_dict["htc"]["output"].update(
        {"esys":[["floater"], ["line1"], ["line2"], ["line3"]] 
         })
    return iea15_H2_dict
    

In [22]:
def generate_turb(iea15_H2_dict, wsp, prepend_str):
    file_name_add = "_turb_%sms"%wsp
    iea15_H2_dict = create_file_names(iea15_H2_dict,file_name_add, prepend_str)
    iea15_H2_dict["htc"]["simulation"]["time_stop"] = 800 
    iea15_H2_dict["htc"]["simulation"]["solvertype"] = 1 # maybe this is needed for turbulence?
    iea15_H2_dict["htc"]["simulation"]["intial_condition"] = 1 

    iea15_H2_dict["htc"]["wind"]["wsp"] = wsp
    iea15_H2_dict["htc"]["wind"]["tint"] = (0.14 * (0.75 * (wsp) + 5.6)) / wsp
    iea15_H2_dict["htc"]["wind"].update({"shear_format": [3, 0.20]}) # need because it isn't a single value
    iea15_H2_dict["htc"]["wind"]["turb_format"] = 1
    iea15_H2_dict["htc"]["wind"]["scale_time_start"] = 200
        
    iea15_H2_dict["htc"]["wind"].update({"mann": 
                                 {"create_turb_parameters": [29.4, 1.0, 3.9, 1001, 1],
                                  "filename_u": ["./turb/turb_l_%sms_29.4_ae_1.0_g_3.9_s1001_hfc_1_u.bin"%wsp],
                                  "filename_v": ["./turb/turb_l_%sms_29.4_ae_1.0_g_3.9_s1001_hfc_1_v.bin"%wsp],
                                  "filename_w": ["./turb/turb_l_%sms_29.4_ae_1.0_g_3.9_s1001_hfc_1_w.bin"%wsp],
                                  "box_dim_u": [8192, 0.87890625],
                                  "box_dim_v": [32, 8.12903],
                                  "box_dim_w": [32, 8.12903]}
                                  })
    iea15_H2_dict["htc"]["wind"]["mann"].add_comment("create_turb_parameters", ["L (length scale), alfa_epsilon, gamma, seed, high freq compensation"])
    iea15_H2_dict["htc"]["wind"]["mann"].add_comment("box_dim_u", ["U*T/n_u"])
    iea15_H2_dict["htc"]["wind"]["mann"].add_comment("box_dim_v", ["wid/(n_v-1), wid = rotdiam + 5%"])
    iea15_H2_dict["htc"]["wind"]["mann"].add_comment("box_dim_w", ["ht/(n_w-1), ht = rotdiam + 5%"])
    return iea15_H2_dict


# Build Simulations

Need to have a certain sequence of commands to get hawc2 files printed which will run.

In [26]:

wind_speeds = [0,3, 5, 7, 9, 10, 10.5, 11, 13, 15, 20, 24] # only for 1_1_1 & 2_1_1 for power curve
#wind_speeds = [4, 8, 10.8, 12, 18, 24] # for rest of steady state
# [4, 6, 8, 10, 10.8, 11, 15, 20, 22, 24] # for step winds?
wsp_turb = [8, 10.8 ,12, 18]

In [27]:
for i, wsp in enumerate(wind_speeds):
    iea15_template = load_iea15_template(template_file,base_filename,iea15_manual_repo_fpath)
    iea15_template.write_hawc2(".") #resets the data files to be used.
    add_damping(iea15_template)
    adj_blade_nbodies(iea15_template)
    adj_blade_output(iea15_template)
    adj_fix_rotor(iea15_template)
    generate_steady_state_wo_ctrl(iea15_template,wsp,"1_1_1_") # change based on design hierachy
    add_aero_sec_int_forces(iea15_template)
    add_blade_pos_sensor(iea15_template)
    add_shaft_force_sensor(iea15_template)
    if i != 0:
        remove_viz_ani(iea15_template) #removes ani viz files except for first instance
    iea15_template.write_hawc2(".")
    

In [28]:
for i, wsp in enumerate(wind_speeds):
    iea15_template = load_iea15_template(template_file,base_filename,iea15_manual_repo_fpath)
    iea15_template.write_hawc2(".") #resets the data files to be used.
    add_damping(iea15_template)
    adj_blade_nbodies(iea15_template)
    adj_blade_output(iea15_template)
    adj_fix_rotor(iea15_template)
    generate_turb(iea15_template, wsp,"1_1_2_") # change based on design hierachy
    add_aero_sec_int_forces(iea15_template)
    add_blade_pos_sensor(iea15_template)
    if i != 0:
        remove_viz_ani(iea15_template) #removes ani viz files except for first instance
    iea15_template.write_hawc2(".")

In [29]:
for i, wsp in enumerate(wind_speeds):
    iea15_template = load_iea15_template(template_file,base_filename,iea15_manual_repo_fpath)
    iea15_template.write_hawc2(".") #resets the data files to be used.
    add_damping(iea15_template)
    adj_blade_nbodies(iea15_template)
    adj_blade_output(iea15_template)
    adj_fix_rotor(iea15_template)
    adj_stiff_bld_new(iea15_template)
    generate_steady_state_wo_ctrl(iea15_template,wsp,"1_2_1_") # change based on design hierachy
    add_aero_sec_int_forces(iea15_template)
    add_blade_pos_sensor(iea15_template)
    if i != 0:
        remove_viz_ani(iea15_template) #removes ani viz files except for first instance
    iea15_template.write_hawc2(".")

In [28]:
for i, wsp in enumerate(wind_speeds):
    iea15_template = load_iea15_template(template_file,base_filename,iea15_manual_repo_fpath)
    iea15_template.write_hawc2(".") #resets the data files to be used.
    add_damping(iea15_template)
    adj_blade_nbodies(iea15_template)
    adj_blade_output(iea15_template)
    adj_fix_rotor(iea15_template)
    adj_stiff_tors_bld_new(iea15_template)
    generate_steady_state_wo_ctrl(iea15_template,wsp,"1_3_1_") # change based on design hierachy
    add_aero_sec_int_forces(iea15_template)
    add_blade_pos_sensor(iea15_template)
    if i != 0:
        remove_viz_ani(iea15_template) #removes ani viz files except for first instance
    iea15_template.write_hawc2(".")

In [30]:
for i, wsp in enumerate(wind_speeds):
    iea15_template = load_iea15_template(template_file,base_filename,iea15_manual_repo_fpath)
    iea15_template.write_hawc2(".") #resets the data files to be used.
    add_damping(iea15_template)
    adj_blade_nbodies(iea15_template)
    adj_blade_output(iea15_template)
    adj_fix_tower_bottom(iea15_template)
    generate_steady_state_wo_ctrl(iea15_template,wsp,"2_1_1_") # change based on design hierachy
    add_aero_sec_int_forces(iea15_template)
    add_blade_pos_sensor(iea15_template)
    add_shaft_force_sensor(iea15_template)
    update_tower_nbodies(iea15_template)
    if i != 0:
        remove_viz_ani(iea15_template) #removes ani viz files except for first instance
    iea15_template.write_hawc2(".")

KeyboardInterrupt: 

In [ ]:
for i, wsp in enumerate(wind_speeds):
    iea15_template = load_iea15_template(template_file,base_filename,iea15_manual_repo_fpath)
    iea15_template.write_hawc2(".") #resets the data files to be used.
    add_damping(iea15_template)
    adj_blade_nbodies(iea15_template)
    adj_blade_output(iea15_template)
    adj_fix_tower_bottom(iea15_template)
    generate_turb(iea15_template, wsp,"2_1_2_") # change based on design hierachy
    add_aero_sec_int_forces(iea15_template)
    add_blade_pos_sensor(iea15_template)
    if i != 0:
        remove_viz_ani(iea15_template) #removes ani viz files except for first instance
    iea15_template.write_hawc2(".")

In [ ]:
for i, wsp in enumerate(wind_speeds):
    iea15_template = load_iea15_template(template_file,base_filename,iea15_manual_repo_fpath)
    iea15_template.write_hawc2(".") #resets the data files to be used.
    add_damping(iea15_template)
    adj_blade_nbodies(iea15_template)
    adj_blade_output(iea15_template)
    generate_steady_state_wo_ctrl(iea15_template,wsp,"3_1_1_") # change based on design hierachy
    add_aero_sec_int_forces(iea15_template)
    add_blade_pos_sensor(iea15_template)
    if i != 0:
        remove_viz_ani(iea15_template) #removes ani viz files except for first instance
    iea15_template.write_hawc2(".")
    

In [ ]:
stepwind = [4]


In [ ]:
for i, wsp in enumerate(stepwind):
    iea15_template = load_iea15_template(template_file,base_filename,iea15_manual_repo_fpath)
    iea15_template.write_hawc2(".") #resets the data files to be used.
    add_damping(iea15_template)
    adj_blade_nbodies(iea15_template)
    adj_blade_output(iea15_template)
    generate_stepwind(iea15_template,wsp,"3_1_3_") # change based on design hierachy
    add_aero_sec_int_forces(iea15_template)
    add_blade_pos_sensor(iea15_template)
    if i != 0:
        remove_viz_ani(iea15_template) #removes ani viz files except for first instance
    iea15_template.write_hawc2(".")

In [ ]:
for i, wsp in enumerate(wind_speeds):
    iea15_template = load_iea15_template(template_file,base_filename,iea15_manual_repo_fpath)
    iea15_template.write_hawc2(".") #resets the data files to be used.
    add_damping(iea15_template)
    adj_blade_nbodies(iea15_template)
    adj_blade_output(iea15_template)
    generate_turb(iea15_template, wsp,"3_1_2_") # change based on design hierachy
    add_aero_sec_int_forces(iea15_template)
    add_blade_pos_sensor(iea15_template)
    if i != 0:
        remove_viz_ani(iea15_template) #removes ani viz files except for first instance
    iea15_template.write_hawc2(".")

In [34]:
for i, wsp in enumerate(wind_speeds):
    iea15_template = load_iea15_template(template_file,base_filename,iea15_manual_repo_fpath)
    iea15_template.write_hawc2(".") #resets the data files to be used.
    add_damping(iea15_template)
    adj_blade_nbodies(iea15_template)
    adj_blade_output(iea15_template)
    generate_steady_state_wo_ctrl(iea15_template,wsp,"4_1_1_") # change based on design hierachy
    add_aero_sec_int_forces(iea15_template)
    add_blade_pos_sensor(iea15_template)
    if i != 0:
        remove_viz_ani(iea15_template) #removes ani viz files except for first instance
    iea15_template.write_hawc2(".")

# Filter Files

Need to do this in run_series.py.
Can test creating the array for the file to run.

In [2]:
res_folder = "htc"
cases = ["1_1_1", 
         "1_1_2",
         "1_2_1",
         "2_1_1",
         "2_1_2",
         "3_1_1",
         "3_1_2"]
ms_filter = [4,8]
filtered_files = functions.filter_files(res_folder,start_patterns=cases,ms_values = ms_filter)
for item in filtered_files:
    print(item)



1_1_1_15MW_UMaineSemi_Fix_rotor_Wo_ctrl_steady_4ms
1_1_1_15MW_UMaineSemi_Fix_rotor_Wo_ctrl_steady_8ms
1_1_2_15MW_UMaineSemi_Fix_rotor_turb_8ms
1_2_1_15MW_UMaineSemi_Fix_rotor_Stiff_bld_Wo_ctrl_steady_4ms
1_2_1_15MW_UMaineSemi_Fix_rotor_Stiff_bld_Wo_ctrl_steady_8ms
2_1_1_15MW_UMaineSemi_Fix_tow_bot_Wo_ctrl_steady_4ms
2_1_1_15MW_UMaineSemi_Fix_tow_bot_Wo_ctrl_steady_8ms
2_1_2_15MW_UMaineSemi_Fix_tow_bot_turb_8ms
3_1_1_15MW_UMaineSemi_Wo_ctrl_steady_4ms
3_1_1_15MW_UMaineSemi_Wo_ctrl_steady_8ms
3_1_2_15MW_UMaineSemi_turb_8ms
